In [7]:
import pandas as pd
import numpy as np
from helpers import calibrate_train_clfs, load_data, get_baseline, print_results, run_clf

In [8]:
keep_columns =  ['IsTraining','FTR', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 
    'away_away_team_corners_against', 'away_away_team_corners_for',	'away_away_team_goals_against',	'away_away_team_goals_for',	
    'away_away_team_possession',	'away_away_team_shotsoff_against',	'away_away_team_shotsoff_for',	'away_away_team_shotson_against',
    	'away_away_team_shotson_for',	'away_direct_team_corners_against',	'away_direct_team_corners_for',	'away_direct_team_goals_against',
        'away_direct_team_goals_for',	'away_direct_team_possession',	'away_direct_team_shotsoff_against',	'away_direct_team_shotsoff_for',
        'away_direct_team_shotson_against',	'away_direct_team_shotson_for',	'home_direct_team_corners_against',	'home_direct_team_corners_for',	
        'home_direct_team_goals_against',	'home_direct_team_goals_for',	'home_direct_team_possession',	'home_direct_team_shotsoff_against',
        'home_direct_team_shotsoff_for',	'home_direct_team_shotson_against',	'home_direct_team_shotson_for',	
        'home_home_team_corners_against',	'home_home_team_corners_for',	'home_home_team_goals_against',	'home_home_team_goals_for',	
        'home_home_team_possession',	'home_home_team_shotsoff_against',	'home_home_team_shotsoff_for',	'home_home_team_shotson_against',
                 'home_home_team_shotson_for','Referee' 
]
 
drop_columns = [
    'B365H', 'B365D', 'B365A',
    'AC', 'AF', 'AR', 'AS', 'AST', 'AY', 
    'BWA', 'BWD', 'BWH', 'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 'BbAv>2.5', 'BbAvA', 'BbAvAHA', 
    'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 'BbMxA', 'BbMxAHA', 'BbMxAHH', 'BbMxD',
    'BbMxH', 'BbOU', 'FTAG', 
    'HC', 'HF', 'HR', 'HS', 'HST', 
    'HTAG', 'HTHG', 'HTR', 'HY', 'IWA', 'IWD', 'IWH', 
    'LBA', 'LBD', 'LBH', 'PSA', 'PSCA', 'PSCD', 'PSCH', 'PSD', 'PSH', 
    'VCA', 'VCD', 'VCH', 'WHA', 'WHD', 'WHH', 'GoalFirstHalf', 'SHHG', 'SHAG', 'GoalSecondHalf','FTR','BothToScore',
]

#
target_column = "FTR"
drop_columns.remove(target_column)
X,y = load_data(columns_to_drop=drop_columns, is_training=True, target_name=target_column)

KEEEP
LENGTH1180


In [9]:
type(y)

pandas.core.series.Series

In [10]:
import numpy
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils
from keras.layers import Bidirectional, LSTM
from keras.initializers import glorot_normal

from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'keras'

In [11]:
y=y.values
X=X.values

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


NameError: name 'LabelEncoder' is not defined

In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(12,  activation='relu')) # input_dim=314,
#   model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1, recurrent_dropout=0.1,kernel_initializer=glorot_normal(seed=None)),name = 'BDLSTM1')) #Best = 300,0.25,0.25
#    model.add(Dense(8, activation='relu'))
#    model.add(Dense(1, activation='linear'))
    model.add(Dense(3, activation='softmax'))
#     # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mae', 'mse', f1_m], verbose=1)
    #model.compile(loss='mse', optimizer='adam', metrics=['mse','mae']) 
    return model
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=2, verbose=1)

In [ ]:
# #seed = 7
# #numpy.random.seed(seed)
# from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)# max_train_size=100
# for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# #kfold = KFold(n_splits=10)
# # shuffle=True, random_state=seed
results = cross_val_score(estimator, X, dummy_y, cv=tscv.split(X))

In [ ]:
results

In [ ]:
import statistics 
x = statistics.mean(results)
x

In [ ]:
X_train_calibrate, X_test, y_train_calibrate, y_test = train_test_split(X, y, test_size=0.20)
X_train, X_calibrate, y_train, y_calibrate = train_test_split(X_train_calibrate, y_train_calibrate, test_size = 0.3, random_state = 42)

In [ ]:
model = baseline_model()
model.fit(X, dummy_y, nb_epoch=30, batch_size=10, validation_split=0.2, verbose=1)

In [ ]:
prediction = model.predict(X[0:100])
prediction /= prediction.sum(axis=1).reshape((-1, 1))
prediction